In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = "/content/drive/My Drive/M2/DeepLearning"

import sys
sys.path.append(path)

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
print(train_images.shape)
print(test_images.shape)


import mnist_reader
X_train, y_train = mnist_reader.load_mnist(path+'/Datasets', kind='train')
X_test, y_test = mnist_reader.load_mnist(path+'/Datasets', kind='t10k')
print(X_train.shape)
print(X_test.shape)

X_train, X_test, train_images, test_images = X_train / 255.0, X_test / 255.0, train_images / 255.0, test_images / 255.0

### **1. Deep Autoencoder**

In [0]:
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 200
batch_size = 256

encoding_dim = 7*7
hidden_dim = 512

activation='elu'
optimizer='adam'
loss='mean_squared_error'
loss='binary_crossentropy'
#################################################

out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
if activation=='leaky_relu':
  activation=None

#################################################
# Auto-encoder
#################################################
input_img = Input(shape=(784,))


####### Encoder #######
#   Layer
encoded = Dense(hidden_dim, activation=activation)(input_img)
if activation is None: encoded = layers.LeakyReLU()(encoded)

#   Layer
encoded = Dense(hidden_dim//4, activation=activation)(encoded)
if activation is None: encoded = layers.LeakyReLU()(encoded)

#   Layer
encoded = Dense(encoding_dim, activation=out_activation)(encoded)


####### Decoder #######
#   Layer
decoded = Dense(hidden_dim//4, activation=activation)(encoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = Dense(hidden_dim, activation=activation)(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = Dense(784, activation=out_activation)(decoded)


####### Make model #######
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=optimizer, loss=loss)

autoencoder.summary()


####### Train #######
autoencoder.fit(X_train, X_train,
                epochs=epoch,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, X_test))


####### Encode images #######
encoder = Model(input_img, encoded)
encoded_images = encoder.predict(X_train)
#################################################



#################################################
# KMeans
#################################################
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, y_train))
print(ari(kmeans.labels_, y_train))
#################################################

### **2. Deep Convolutional Autoencoder**

In [0]:
from tensorflow import keras
from tensorflow.keras import Input, layers, models
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from math import sqrt

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 100
batch_size = 256

encoding_dim = 7*7

#activation='elu'
activation='leaky_relu'
optimizer='adam'
#loss='mean_squared_error'
loss='binary_crossentropy'
#################################################

encoding_size = int(sqrt(encoding_dim))
out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
if activation=='leaky_relu':
  activation=None

#################################################
# Convolutional Auto-encoder
#################################################
input_img = Input((28, 28, 1))


####### Encoder #######
#   Layer
encoded = layers.Conv2D(32, (3, 3), activation=activation)(input_img)
if activation is None: encoded = layers.LeakyReLU()(encoded)
encoded = layers.MaxPooling2D((2, 2))(encoded)

#   Layer
encoded = layers.Conv2D(64, (3, 3), activation=activation)(encoded)
if activation is None: encoded = layers.LeakyReLU()(encoded)
encoded = layers.MaxPooling2D((2, 2))(encoded)

#   Layer
encoded = layers.Conv2D(64, (3, 3), activation=activation)(encoded)
if activation is None: encoded = layers.LeakyReLU()(encoded)

#   Layer
encoded = layers.Flatten()(encoded)
encoded = layers.Dense(encoding_dim, activation=out_activation)(encoded)


####### Decoder #######
decoded = layers.Reshape((encoding_size,encoding_size,1))(encoded)
#   Layer
decoded = layers.Conv2DTranspose(64,(3, 3), strides=2, activation=activation, padding='same')(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = layers.Conv2DTranspose(64,(3, 3), strides=2, activation=activation, padding='same')(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = layers.Conv2DTranspose(32,(3, 3), activation=activation, padding='same')(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = layers.Conv2D(1, (3, 3), activation=out_activation, padding='same')(decoded)


####### Make model #######
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer=optimizer, loss=loss)

autoencoder.summary()


####### Train #######
autoencoder.fit(train_images, train_images,
                epochs=epoch,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(test_images, test_images))


####### Encode images #######
encoder = Model(input_img, encoded)
encoded_images = encoder.predict(train_images)
print(encoded_images.shape)
#################################################



#################################################
# KMeans
#################################################
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, train_labels))
print(ari(kmeans.labels_, train_labels))
#################################################

### **3. Deep Convolutional Adversarial Autoencoder**

In [0]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers, models, Input
import numpy as np
from statistics import mean

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)



#################################################
# Hyper-parameters
#################################################
batch_size = 256

epoch = 50
noise_dim = 100
#################################################



#################################################
# Generator
#################################################
def make_generator_model():
    input_img = Input((100, ))
    layer = layers.Dense(7*7*256, use_bias=False)(input_img)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Reshape((7, 7, 256))(layer)
    #assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    layer = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(layer)
    #assert model.output_shape == (None, 7, 7, 128)(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(layer)
    #assert model.output_shape == (None, 14, 14, 64)(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(layer)
    #assert model.output_shape == (None, 28, 28, 1)

    model = models.Model(input_img, layer)

    return model

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
#################################################



#################################################
# Discriminator
#################################################
def make_discriminator_model():
    input_img = Input((28, 28, 1))
    layer = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',)(input_img)
    layer = layers.LeakyReLU()(layer)
    layer = layers.Dropout(0.3)(layer)

    layer = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(layer)
    layer = layers.LeakyReLU()(layer)
    layer = layers.Dropout(0.3)(layer)

    layer = layers.Flatten()(layer)
    layer = layers.Dense(1)(layer)

    model = models.Model(input_img, layer)

    return model

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
#################################################    

discriminator = make_discriminator_model()
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

generator = make_generator_model()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset):
  for ep in range(epoch):
    print("epoch : ",ep, " / ", epoch)
    gen_losses, disc_losses = [], []
    for it in range(int(dataset.shape[0] / batch_size)):
      image_batch = dataset[np.random.randint(low=0,high=dataset.shape[0],size=batch_size)]
      gen_loss, disc_loss = train_step(image_batch)
      gen_losses.append(gen_loss.numpy())
      disc_losses.append(disc_loss.numpy())

    print("Generator loss     =",mean(gen_losses), "\nDiscriminator loss =",mean(disc_losses))

train(train_images)